### Step 1: Split the data and create the evaluator objects

In [5]:
from implicit.gpu.als import AlternatingLeastSquares
import implicit

print("CUDA support in Implicit:", implicit.gpu.CUDA_AVAILABLE)

AttributeError: module 'implicit.gpu' has no attribute 'CUDA_AVAILABLE'

In [6]:
import torch
print(torch.cuda.is_available())

True


In [7]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("RECSYS_POLIMI_2024"), '..'))

from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

URM_all_dataframe = pd.read_csv(filepath_or_buffer="data_train.csv")
ICM_all = pd.read_csv(filepath_or_buffer="data_ICM_metadata.csv")
URM_all_dataframe.head(n=10)


URM_train_validation, URM_test = train_test_split(URM_all_dataframe, test_size=0.2, random_state=42)
URM_train, URM_validation = train_test_split(URM_train_validation, test_size=0.2, random_state=42)

# # Reset indices for cleaner output (optional)
# URM_test = URM_test.reset_index(drop=True)
# URM_validation = URM_validation.reset_index(drop=True)
# URM_train = URM_train.reset_index(drop=True)
# URM_train_validation = URM_train_validation.reset_index(drop=True)

# Display the results
print("URM_train (80% of URM_ALL):")
print(URM_train.head(n=10))

print("\nURM_validation (20% of URM_ALL):")
print(URM_validation.head(n=10))

print("\nURM_test (20% of URM_ALL):")
print(URM_test.head(n=10))


URM_train (80% of URM_ALL):
         user_id  item_id  data
136935      1523    11379   1.0
286638      3415    29647   1.0
499343      6571    11101   1.0
1612162    31036    17962   1.0
1231767    22131    20291   1.0
1419045    25670    32878   1.0
505970      6677    30720   1.0
1309905    23612    35978   1.0
1026162    17411     9221   1.0
991787     16720     2668   1.0

URM_validation (20% of URM_ALL):
         user_id  item_id  data
1293463    23345    37683   1.0
978469     16471     1559   1.0
1139447    19988     6956   1.0
628101      8833      614   1.0
1276102    23100    20305   1.0
570596      7748      117   1.0
1232901    22148    35968   1.0
1544801    29212    33031   1.0
1033303    17563    34624   1.0
658881      9451     9732   1.0

URM_test (20% of URM_ALL):
         user_id  item_id  data
1092502    18939     8673   1.0
367273      4669     2436   1.0
1181580    20972    25295   1.0
1243868    22394    23927   1.0
302903      3680    24294   1.0
1337700    240

In [8]:
URM_all_dataframe = csr_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_train = csr_matrix((URM_train['data'], (URM_train['user_id'], URM_train['item_id'])))
URM_validation = csr_matrix((URM_validation['data'], (URM_validation['user_id'], URM_validation['item_id'])))
URM_test = csr_matrix((URM_test['data'], (URM_test['user_id'], URM_test['item_id'])))
URM_train_validation = csr_matrix((URM_train_validation['data'], (URM_train_validation['user_id'], URM_train_validation['item_id'])))
ICM_all = csr_matrix((ICM_all['data'], (ICM_all['item_id'], ICM_all['feature_id'])))

#evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 401 ( 1.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 152 ( 0.4%) Users that have less than 1 test interactions


In [9]:
import implicit.gpu
print(implicit.gpu.HAS_CUDA)  # Dovrebbe restituire True

False


In [14]:
from Recommenders.MatrixFactorization.IALSJnoearly import ImplicitALSRecommender
model = ImplicitALSRecommender(URM_train_validation)
model.fit( epochs=6,
                    num_factors=1650,
                    confidence_scaling="linear",
                    alpha=6,
                    epsilon=0.1,
                    reg=100,
                    use_gpu=False,
                    num_threads=11,
                    calculate_training_loss=True
                    )
result_df, _ = evaluator_test.evaluateRecommender(model)
print(result_df.loc[10]["MAP"])

Using gpu: False


100%|██████████| 6/6 [03:17<00:00, 32.87s/it, loss=0.00634]


EvaluatorHoldout: Processed 35584 (100.0%) in 21.55 sec. Users per second: 1651
0.023205030841397897


In [15]:
import optuna
import pandas as pd
def objective_function(optuna_trial):
    
    als = ImplicitALSRecommender(URM_train_validation)
    als.fit( epochs=6,
                    num_factors=optuna_trial.suggest_int("num_factors", 1600, 1700),
                    confidence_scaling="linear",
                    alpha=optuna_trial.suggest_float("alpha", 5, 7),
                    epsilon=optuna_trial.suggest_float("epsilon", 0, 1),
                    reg=optuna_trial.suggest_float("reg", 0, 10),
                    use_gpu=False,
                    num_threads=8,
                    calculate_training_loss=True
                    )
    
    result_df, _ = evaluator_test.evaluateRecommender(als)
    return result_df.loc[10]["MAP"]

### Step 3: Create an object/function that will be called after every call of the objective function to log the results in a dataframe for easier readability

In [16]:
class SaveResults(object):
    def __init__(self, save_path="optuna_progressIALSilritorno.csv"):
        # File dove salvare i risultati
        self.save_path = save_path

        # Se esiste già un file, carichiamo i dati per continuare
        try:
            self.results_df = pd.read_csv(self.save_path)
        except FileNotFoundError:
            self.results_df = pd.DataFrame(columns=["trial_number", "value", "params"])
    
    def __call__(self, optuna_study, optuna_trial):
        # Salva i risultati del trial corrente
        trial_data = {
            "trial_number": optuna_trial.number,
            "value": optuna_trial.value,
            "params": optuna_trial.params
        }

        # Aggiungi i risultati al DataFrame
        self.results_df = pd.concat([self.results_df, pd.DataFrame([trial_data])], ignore_index=True)

        # Salva i risultati progressivamente su disco
        self.results_df.to_csv(self.save_path, index=False)

        # Salva anche il miglior risultato corrente
        best_trial = optuna_study.best_trial
        best_result = {
            "trial_number": best_trial.number,
            "value": best_trial.value,
            **best_trial.params
        }

        best_results_df = pd.DataFrame([best_result])
        best_results_df.to_csv("best_optuna_resultsIALSREGilritorno.csv", index=False)
        
        print(f"Results saved to {self.save_path} and best results to 'best_optuna_results.csv'")


In [17]:
# Creazione dello studio Optuna
optuna_study = optuna.create_study(direction="maximize")

# Inizializzazione del callback per salvare progressivamente i risultati
save_results = SaveResults(save_path="optuna_progressIALSilritorno.csv")

# Esecuzione dell'ottimizzazione
optuna_study.optimize(
    objective_function,
    callbacks=[save_results],
    n_trials=40
)

[I 2024-12-29 19:30:00,040] A new study created in memory with name: no-name-b954342f-c576-4e3e-9a06-ea1bee049d3a


Using gpu: False


100%|██████████| 6/6 [03:39<00:00, 36.64s/it, loss=0.00233]


EvaluatorHoldout: Processed 35584 (100.0%) in 21.47 sec. Users per second: 1658


[I 2024-12-29 19:34:02,046] Trial 0 finished with value: 0.048748451015971375 and parameters: {'num_factors': 1663, 'alpha': 5.905668644028478, 'epsilon': 0.6494927521373227, 'reg': 2.850440868205312}. Best is trial 0 with value: 0.048748451015971375.


...

Results saved to optuna_progressIALSilritorno.csv and best results to 'best_optuna_results.csv'
Using gpu: False


100%|██████████| 6/6 [03:39<00:00, 36.59s/it, loss=0.00257]


EvaluatorHoldout: Processed 35584 (100.0%) in 21.76 sec. Users per second: 1635


[I 2024-12-29 22:15:49,019] Trial 39 finished with value: 0.04902107889581031 and parameters: {'num_factors': 1667, 'alpha': 6.557731210128585, 'epsilon': 0.3791070647315319, 'reg': 5.389443028322923}. Best is trial 27 with value: 0.049124936657814804.


Results saved to optuna_progressIALSilritorno.csv and best results to 'best_optuna_results.csv'


In [18]:
optuna_study.best_trial.params

{'num_factors': 1684,
 'alpha': 6.524580033299327,
 'epsilon': 0.556377225083557,
 'reg': 6.399046570327465}

In [20]:
als = ImplicitALSRecommender(URM_all_dataframe)
als.fit( epochs=6,
                    confidence_scaling="linear",
                    use_gpu=False,
                    num_threads=8,
                    calculate_training_loss=True,
                        **optuna_study.best_trial.params
                    )

Using gpu: False


100%|██████████| 6/6 [03:54<00:00, 39.03s/it, loss=0.00307]


In [21]:
recommendations_list = []
user = pd.read_csv(filepath_or_buffer="data_target_users_test.csv")

# Loop through each user and get the top 10 recommendations
for user_id in user["user_id"]:  # assuming URM.shape[0] is the total number of users
    top_10_recommendations = als.recommend(user_id, cutoff=10, remove_seen_flag=True)
    # Join recommendations into a space-separated string
    item_list_str = ' '.join(map(str, top_10_recommendations))
    recommendations_list.append([user_id, item_list_str])
# Convert the list to a DataFrame
recommendations_df = pd.DataFrame(recommendations_list, columns=['user_id', 'item_list'])

# Display the resulting table
print(recommendations_df.head(2))

# Optionally, save to CSV
recommendations_df.to_csv('top_10_recommendationsalsilritorno.csv', index=False)

   user_id                                          item_list
0        0  11699 531 14931 1425 9911 8685 3152 11542 1510...
1        1  13733 14748 120 9911 3146 13006 11875 8685 119...


In [27]:
from Recommenders.MatrixFactorization.IALSJ import ImplicitALSRecommender

for i in range(20):
        print("num_factors: ", 140+i*10)
        model = ImplicitALSRecommender(URM_train_validation)
        model.fit( epochs=30,
                num_factors=140+i*10,
                confidence_scaling="linear",
                alpha=1.0,
                epsilon=1.0,
                reg=1e-4,
                use_gpu=False,
                num_threads=8,
                calculate_training_loss=True
                )
        result_df, _ = evaluator_test.evaluateRecommender(model)
        print(result_df.loc[10]["MAP"])

num_factors:  140
Using gpu: False


100%|██████████| 30/30 [00:57<00:00,  1.92s/it, loss=0.00175]


EvaluatorHoldout: Processed 35584 (100.0%) in 20.35 sec. Users per second: 1748
0.02678637893256766
num_factors:  150
Using gpu: False


100%|██████████| 30/30 [01:14<00:00,  2.50s/it, loss=0.00174]


EvaluatorHoldout: Processed 35584 (100.0%) in 20.39 sec. Users per second: 1745
0.027462935901278417
num_factors:  160
Using gpu: False


...

EvaluatorHoldout: Processed 35584 (100.0%) in 21.49 sec. Users per second: 1656
0.0337946004803291
num_factors:  320
Using gpu: False


100%|██████████| 30/30 [13:08<00:00, 26.27s/it, loss=0.0016] 


EvaluatorHoldout: Processed 35584 (100.0%) in 21.54 sec. Users per second: 1652
0.03412420063948748
num_factors:  330
Using gpu: False


100%|██████████| 30/30 [09:02<00:00, 18.09s/it, loss=0.00159]


EvaluatorHoldout: Processed 35584 (100.0%) in 22.30 sec. Users per second: 1596
0.03436945086787614


In [38]:
from Recommenders.MatrixFactorization.IALSJ import ImplicitALSRecommender

for i in range(40):
        print("num_factors: ", 550+i*50)
        model = ImplicitALSRecommender(URM_train_validation)
        model.fit( epochs=30,
                num_factors=550+i*50,
                confidence_scaling="linear",
                alpha=1.0,
                epsilon=1.0,
                reg=1e-4,
                use_gpu=False,
                num_threads=8,
                calculate_training_loss=True
                )
        result_df, _ = evaluator_test.evaluateRecommender(model)
        print(result_df.loc[10]["MAP"])

num_factors:  550
Using gpu: False


100%|██████████| 30/30 [11:14<00:00, 22.48s/it, loss=0.00146]


EvaluatorHoldout: Processed 35584 (100.0%) in 23.43 sec. Users per second: 1519
0.03819559642464449
num_factors:  600
Using gpu: False


100%|██████████| 30/30 [11:36<00:00, 23.22s/it, loss=0.00144]


EvaluatorHoldout: Processed 35584 (100.0%) in 24.60 sec. Users per second: 1447
0.0388256035796351
num_factors:  650
Using gpu: False


...

EvaluatorHoldout: Processed 35584 (100.0%) in 30.51 sec. Users per second: 1166
0.04564001365871425
num_factors:  2050
Using gpu: False


100%|██████████| 30/30 [32:51<00:00, 65.73s/it, loss=0.000964]


EvaluatorHoldout: Processed 35584 (100.0%) in 32.63 sec. Users per second: 1091
0.04573662833126206
num_factors:  2100
Using gpu: False


 27%|██▋       | 8/30 [10:02<27:38, 75.37s/it, loss=0.000962]


KeyboardInterrupt: 

In [4]:
from Recommenders.MatrixFactorization.IALSJ import ImplicitALSRecommender
for i in range(10):
    print("alpha: ", 1+i)
    model = ImplicitALSRecommender(URM_train)
    model.fit( epochs=15,
                    num_factors=1650,
                    confidence_scaling="linear",
                    alpha=1+i,
                    epsilon=1.0,
                    reg=1e-4,
                    use_gpu=False,
                    num_threads=8,
                    calculate_training_loss=True
                    )
    result_df, _ = evaluator_test.evaluateRecommender(model)
    print(result_df.loc[10]["MAP"])
    result_df, _ = evaluator_validation.evaluateRecommender(model)
    print(result_df.loc[10]["MAP"])

alpha:  1
Using gpu: False


100%|██████████| 15/15 [11:50<00:00, 47.37s/it, loss=0.000873]


EvaluatorHoldout: Processed 35584 (100.0%) in 29.78 sec. Users per second: 1195
0.033510330128753166
EvaluatorHoldout: Processed 35335 (100.0%) in 29.93 sec. Users per second: 1181
0.025917308482752786
alpha:  2
Using gpu: False


100%|██████████| 15/15 [12:12<00:00, 48.84s/it, loss=0.00113]


EvaluatorHoldout: Processed 35584 (100.0%) in 29.29 sec. Users per second: 1215
0.03486536217454494
EvaluatorHoldout: Processed 35335 (100.0%) in 30.10 sec. Users per second: 1174
0.02717416518987126
alpha:  3
Using gpu: False


...

EvaluatorHoldout: Processed 35584 (100.0%) in 27.99 sec. Users per second: 1271
0.0360233052397355
EvaluatorHoldout: Processed 35335 (100.0%) in 28.01 sec. Users per second: 1262
0.027993094440737602
alpha:  9
Using gpu: False


100%|██████████| 15/15 [17:50<00:00, 71.35s/it, loss=0.00197]


EvaluatorHoldout: Processed 35584 (100.0%) in 30.18 sec. Users per second: 1179
0.03591523879110879
EvaluatorHoldout: Processed 35335 (100.0%) in 28.85 sec. Users per second: 1225
0.027870495776254087
alpha:  10
Using gpu: False


100%|██████████| 15/15 [12:01<00:00, 48.11s/it, loss=0.00203]


EvaluatorHoldout: Processed 35584 (100.0%) in 30.36 sec. Users per second: 1172
0.035993649277005595
EvaluatorHoldout: Processed 35335 (100.0%) in 29.77 sec. Users per second: 1187
0.027914886090277893


In [5]:
from Recommenders.MatrixFactorization.IALSJ import ImplicitALSRecommender
model = ImplicitALSRecommender(URM_train_validation)
model.fit( epochs=15,
                    num_factors=1650,
                    confidence_scaling="linear",
                    alpha=6,
                    epsilon=1.0,
                    reg=1e-4,
                    use_gpu=False,
                    num_threads=8,
                    calculate_training_loss=True
                    )
result_df, _ = evaluator_test.evaluateRecommender(model)
print(result_df.loc[10]["MAP"])
result_df, _ = evaluator_validation.evaluateRecommender(model)
print(result_df.loc[10]["MAP"])

Using gpu: False


100%|██████████| 15/15 [12:15<00:00, 49.06s/it, loss=0.00208]


EvaluatorHoldout: Processed 35584 (100.0%) in 30.30 sec. Users per second: 1174
0.0481005864950879
EvaluatorHoldout: Processed 35335 (100.0%) in 28.04 sec. Users per second: 1260
0.0
